In [2]:
import pandas as pd

In [7]:
file_paths = ['Large Model Questions.xlsx']

for file_path in file_paths:
    print("-"*50)
    print(file_path)
    print("-"*50)
    for sheet_name in ['PS1','PS2','PS3','PS4','PS5']:
        print(sheet_name)
        df = pd.read_excel(file_path, sheet_name=sheet_name)
        df.columns = df.columns.str.strip()
        df['Prompt'] = df['Prompt'].ffill()
        df['Model'] = df['Model'].ffill()
    
        print()
        avg_similarity_12 = df.groupby('Model')['sim_1_2'].mean().sort_values(ascending=False)
        avg_similarity_13 = df.groupby('Model')['sim_1_3'].mean().sort_values(ascending=False)
        avg_similarity_14 = df.groupby('Model')['sim_1_4'].mean().sort_values(ascending=False)
        avg_similarity_15 = df.groupby('Model')['sim_1_5'].mean().sort_values(ascending=False)
        avg_similarity_16 = df.groupby('Model')['sim_1_6'].mean().sort_values(ascending=False)
        avg_similarity_23 = df.groupby('Model')['sim_2_3'].mean().sort_values(ascending=False)
        avg_similarity_24 = df.groupby('Model')['sim_2_4'].mean().sort_values(ascending=False)
        avg_similarity_25 = df.groupby('Model')['sim_2_5'].mean().sort_values(ascending=False)
        avg_similarity_26 = df.groupby('Model')['sim_2_6'].mean().sort_values(ascending=False)
        avg_similarity_34 = df.groupby('Model')['sim_3_4'].mean().sort_values(ascending=False)
        avg_similarity_35 = df.groupby('Model')['sim_3_5'].mean().sort_values(ascending=False)
        avg_similarity_36 = df.groupby('Model')['sim_3_6'].mean().sort_values(ascending=False)
        avg_similarity_45 = df.groupby('Model')['sim_4_5'].mean().sort_values(ascending=False)
        avg_similarity_46 = df.groupby('Model')['sim_4_6'].mean().sort_values(ascending=False)
        avg_similarity_56 = df.groupby('Model')['sim_5_6'].mean().sort_values(ascending=False)
        
        print(avg_similarity_12, avg_similarity_13, avg_similarity_14, avg_similarity_15, avg_similarity_16, avg_similarity_23, avg_similarity_24, avg_similarity_25, avg_similarity_26, avg_similarity_34, avg_similarity_35, avg_similarity_36, avg_similarity_45, avg_similarity_46, avg_similarity_56, sep="\n\n")
        print("-"*50)

--------------------------------------------------
Large Model Questions.xlsx
--------------------------------------------------
PS1

Model
Mistral_7B    0.912719
Llama2_70B    0.908100
GPT4          0.895762
Palm2         0.893064
GPT3.5        0.892267
Name: sim_1_2, dtype: float64

Model
Mistral_7B    0.892200
GPT4          0.889645
Llama2_70B    0.884530
GPT3.5        0.870746
Palm2         0.870107
Name: sim_1_3, dtype: float64

Model
Mistral_7B    0.898852
Llama2_70B    0.892920
GPT4          0.886239
Palm2         0.871303
GPT3.5        0.868638
Name: sim_1_4, dtype: float64

Model
Palm2         0.894294
Mistral_7B    0.893972
Llama2_70B    0.892366
GPT3.5        0.874869
GPT4          0.869723
Name: sim_1_5, dtype: float64

Model
Llama2_70B    0.880172
Mistral_7B    0.877864
GPT4          0.871640
GPT3.5        0.864536
Palm2         0.855255
Name: sim_1_6, dtype: float64

Model
Mistral_7B    0.899419
Palm2         0.890346
Llama2_70B    0.887645
GPT4          0.885918
GPT3.5  

In [17]:
import pandas as pd
from openpyxl import Workbook
from openpyxl.styles import Alignment, Font, PatternFill, Border, Side
from openpyxl.utils import get_column_letter

# -------- settings --------
src_path = "Small Model Question (1).xlsx"
dst_path = "BERTScore_Summary_Small.xlsx"
sheets = ["PS1", "PS2", "PS3", "PS4", "PS5"]
pairs = ["1_2","1_3","1_4","1_5","1_6","2_3","2_4","2_5","2_6","3_4","3_5","3_6","4_5","4_6","5_6"]
pair_labels = {p: p.replace("_","") for p in pairs}
# --------------------------

def prep(df):
    df = df.copy()
    df.columns = df.columns.str.strip()
    if "Prompt" in df: 
        df["Prompt"] = df["Prompt"].ffill()
    df["Model"] = df["Model"].ffill()
    return df

def per_pair_means(df):
    out = {}
    for p in pairs:
        col = f"sim_{p}"
        s = df.groupby("Model")[col].mean()
        out[p] = s
    return out

# load sheets
dfs = {sn: prep(pd.read_excel(src_path, sheet_name=sn)) for sn in sheets}
df_total = prep(pd.concat(dfs.values(), ignore_index=True))
dfs["Total"] = df_total

means = {sn: per_pair_means(df) for sn, df in dfs.items()}

all_models = pd.Index(
    pd.concat([df["Model"] for df in dfs.values()]).dropna().unique()
)
n_models = len(all_models)

wb = Workbook()
ws = wb.active
ws.title = "Summary"

top_row = 1
for j, p in enumerate(pairs):
    c0 = j*2 + 1
    ws.merge_cells(start_row=top_row, start_column=c0,
                   end_row=top_row, end_column=c0+1)
    cell = ws.cell(row=top_row, column=c0, value=pair_labels[p])

# blocks for PS1..PS5 + Total
block_height = 2 + n_models + 1  # block title + header + models + spacer
for b, sheet_name in enumerate(list(sheets) + ["Total"]):
    start_r = 2 + b*block_height
    for j, p in enumerate(pairs):
        c0 = j*2 + 1

        # Block title
        ws.merge_cells(start_row=start_r, start_column=c0,
                       end_row=start_r, end_column=c0+1)
        cell = ws.cell(row=start_r, column=c0, value=sheet_name)

        # Data rows
        s = means[sheet_name][p]
        r = start_r + 2
        for model, score in s.items():
            ws.cell(row=r, column=c0, value=model)
            ws.cell(row=r, column=c0+1, value=score)
            r += 1

# save file
wb.save(dst_path)
print(f"Done. Wrote {dst_path}")

Done. Wrote BERTScore_Summary_Small.xlsx
